In [1]:
import sys, json, os, ast
import copy
import numpy as np
import pandas as pd
from smart_open import open
from tqdm import tqdm
import pickle as pkl

sys.path.insert(1, "../..")
from src.logger import make_logger
from src.dataloader import TabularDataloader
from src.Trainer import LGBMTrainer, TFTrainer
from src.preprocess import Preprocess

from rdsutils.feature_selection import mrmr
from rdsutils.woe import WOE_Transform
from _utils.feature_selection import feature_selection as fs
from _utils.performance_eval import performance_eval_v3 as p_eval
from rdsutils.feature_selection import FeatureSelector as general_purpose_fsel
from src.feature_selection import FeatureSelector  # to be moved to rdsutils

# new modules
from _utils.sample_weights import get_sample_weight

%load_ext autoreload
%autoreload 2

In [2]:
weight = "weight"
seed = 42

with open("config.json", "r") as f:
    config = json.load(f)
    
display(config.keys()) 

gen3_features = config["data_columns"]["gen3_features"]
gen3_params = config["model_params"]["gen3_params"]
if "scale_pos_weight" in gen3_params:
    del gen3_params["scale_pos_weight"]

bureau_fts = config["data_columns"]["bureau_features_cols"] 
cat_fts = ['t11_t3d_segid', 't11_t3d_segid_supp'] # config["data_columns"]["cat_cols"] 
prescreen_fts = bureau_fts + cat_fts

dict_keys(['data', 'meta', 'data_columns', 'model_params', 'model_features', 'impute_vals', 'monotone'])

In [3]:
display(config["data"]["clean"].keys())

dict_keys(['all_features_dev1', 'all_features_dev2', 'all_features_oot1', 'all_features_oot2', 'subset_dev1', 'subset_dev2'])

#### load data and features

In [4]:
%%time
# data dict
exp_dict = pd.read_csv(config["meta"]["exp_dict_path"])

# fsel data - sampled
dl1 = TabularDataloader(train_path=config["data"]["clean"]["subset_dev1"])
dl1.load_data(debug_size=10000, random_state=seed)

debug_df1, _, _ = dl1.get_data(debug=True)
train_df1, _, _ = dl1.get_data(debug=False)
display(train_df1.shape, debug_df1.shape)

dl2 = TabularDataloader(train_path=config["data"]["clean"]["subset_dev2"])
dl2.load_data(debug_size=10000, random_state=seed)

debug_df2, _, _ = dl2.get_data(debug=True)
train_df2, _, _ = dl2.get_data(debug=False)
display(train_df2.shape, debug_df2.shape)

(228188, 5131)

(10000, 5131)

(215815, 5131)

(10000, 5131)

CPU times: user 1min 6s, sys: 3min 23s, total: 4min 30s
Wall time: 1min 37s


In [24]:
target = "target"
target_indeterminate = "indeterminate"
ri_weight = "weight_ri"

train_df1[target] = train_df1["target_v1"]
train_df1[target_indeterminate] = train_df1["indeterminate_v1"]
train_df1[ri_weight] = train_df1["weight_ri_v1"]

train_df2[target] = train_df2["target_v2"]
train_df2[target_indeterminate] = train_df2["indeterminate_v2"]
train_df2[ri_weight] = train_df2["weight_ri_v2"]

In [25]:
train_df = pd.concat([train_df1, train_df2], axis=0)
train_df.reset_index(drop=True, inplace=True)
train_df.shape

(444003, 5134)

#### get sample weights

In [26]:
col = "ri_source"
weights = {"booked": 1,
           "proxy": 1,
           "others": 0.25}

assert sorted(train_df[col].unique().tolist()) == sorted(list(weights.keys()))

pp = Preprocess(exp_dict)
train_df["weight_eval"] = train_df["weight_cob"] * train_df[ri_weight]
train_df = pp.transform(train_df, prescreen_fts, weights, 
                        drop_indeterminate=target_indeterminate, 
                        existing_weights_col="weight_eval")


100%|██████████| 4205/4205 [00:13<00:00, 311.32it/s]



        added columns:
            weight: training sample_weight scaled using provided weights by ri_source
                weight_eval * weight_sample
        
dropping indeterminate col: indeterminate


In [27]:
# look at weights
display(train_df[["weight", "ri_source"]].groupby("ri_source")["weight"].sum())
display(train_df[["weight_eval", "ri_source"]].groupby("ri_source")["weight_eval"].sum())

ri_source
booked    26191.500
others    44596.625
proxy     38240.500
Name: weight, dtype: float64

ri_source
booked     26191.5
others    178386.5
proxy      38240.5
Name: weight_eval, dtype: float64

#### feature selection

In [28]:
print(target, weight)
train_df.shape, train_df[target].shape, train_df[weight].shape

target weight


((440466, 5137), (440466,), (440466,))

In [29]:
%%time
# %%capture record

nr_to_consider = 200
nr_to_select = 200
fsel_dir = "./artifacts/dev_combined_fsel_v2"

fsel = FeatureSelector(train_df, data_dict=exp_dict)
rankings = fsel.run(prescreen_fts, target, weight, nr_to_consider, nr_to_select,
                    output_dir=fsel_dir, filter_by_logic_expn=True)

# with open("./artifacts/dev1_fsel_1/log.txt", "w") as f:
#     f.write(record.stdout)

target_col: target
weight_col: weight
Preprocessing... generating iv and shaps
prepping woe...
Attrs removed--missing pct>99%:   ['p13_all8162', 'p13_all8163', 'p13_all8380', 'p13_all8723', 'p13_all9222', 'p13_all9223', 'p13_all9230', 'p13_all9239', 'p13_all9240', 'p13_all9249', 'p13_all9260', 'p13_all9280', 'p13_aua8162', 'p13_aua8163', 'p13_bca0401', 'p13_bca5021', 'p13_bca6201', 'p13_col8194', 'p13_hlc5021', 'p13_iln0403', 'p13_mtf8169', 'p13_mtf8656', 'p13_mts8151', 'p13_rpm5020', 'p13_rpm5320', 'p13_rpm5820', 'p13_rpm6160', 'p13_rpm7110', 'p13_rti5020', 'p13_rti5320', 'p13_rti5820', 'p13_uti5030', 'p13_uti5530', 'p13_uti8151', 't11_tall1412', 't11_tall1413', 't11_tall2412', 't11_tcol2556', 't11_tcol2567', 't11_tmti0451', 't11_tmti0452', 't11_tmti0453', 't11_tmti0454', 't11_tmti0455', 't11_tmti0456', 't11_tmti0457', 't11_tmti0458', 't11_tstu0909']
processed  4157  num attributes



100%|██████████| 7/7 [00:01<00:00,  5.83it/s]


prepping lgbm shap
[LightGBM] [Warning] min_data_in_leaf is set=300, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=300
[LightGBM] [Warning] boosting is set=gbdt, boosting_type=gbdt will be ignored. Current value: boosting=gbdt


100%|██████████| 7/7 [00:01<00:00,  5.88it/s]


prepping lgbm mc shap
p13_all9130 1 no monotonic direction - probably should filter out
p13_all9134 1 no monotonic direction - probably should filter out
p13_all9135 1 no monotonic direction - probably should filter out
p13_all9138 1 no monotonic direction - probably should filter out
p13_all9139 1 no monotonic direction - probably should filter out
p13_all9140 1 no monotonic direction - probably should filter out
p13_all9141 1 no monotonic direction - probably should filter out
p13_all9144 1 no monotonic direction - probably should filter out
p13_all9145 1 no monotonic direction - probably should filter out
p13_all9148 1 no monotonic direction - probably should filter out
p13_all9149 1 no monotonic direction - probably should filter out
p13_all9171 1 no monotonic direction - probably should filter out
p13_all9177 1 no monotonic direction - probably should filter out
p13_all9178 1 no monotonic direction - probably should filter out
p13_all9180 1 no monotonic direction - probably should

...

filtering features by logic - experian
dropping 530 features : kept 3675 features
    reason:  not AA
160 features with greater than                 0.95 missing values
dropping 160 features : kept 3515 features
    reason:  too many missing
dropping 585 features : kept 2930 features
    reason:  low_iv
running many to few


100%|██████████| 7/7 [00:01<00:00,  5.58it/s]


saving ranking.csv
running fsel on few
[LightGBM] [Warning] min_data_in_leaf is set=300, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=300
[LightGBM] [Warning] boosting is set=gbdt, boosting_type=gbdt will be ignored. Current value: boosting=gbdt
p13_all9130 1 no monotonic direction - probably should filter out
p13_all9134 1 no monotonic direction - probably should filter out
p13_all9135 1 no monotonic direction - probably should filter out
p13_all9138 1 no monotonic direction - probably should filter out
p13_all9139 1 no monotonic direction - probably should filter out
p13_all9140 1 no monotonic direction - probably should filter out
p13_all9141 1 no monotonic direction - probably should filter out
p13_all9144 1 no monotonic direction - probably should filter out
p13_all9145 1 no monotonic direction - probably should filter out
p13_all9148 1 no monotonic direction - probably should filter out
p13_all9149 1 no monotonic direction - probably should filter out
p13

100%|██████████| 7/7 [00:01<00:00,  5.71it/s]

saving ranking.csv
CPU times: user 3h 31min 18s, sys: 45min 51s, total: 4h 17min 9s
Wall time: 2h 17min 53s


##### get fsel results

* run `fsel.get_rankings(True)` to get ranking_df of features that is ever selected.


##### if computation already made, we can just load it

```python
# initialte project and load back
fsel = FeatureSelector(train_df, data_dict=exp_dict)
fsel.load_state_dict(fsel_dir)
```

##### this is the logic underneath fsel.run

```python
# setup
features = prescreen_fts
target_col = target
weight_col = weight
output_dir = fsel_dir
corr_threshold = 0.8
filter_by_logic_expn = True

# first preprocessing
fsel.preprocess(features, target_col, weight_col, output_dir=output_dir)

if filter_by_logic_expn:
    print("filtering features by logic - experian")
    features = fsel.filter_by_logic_expn(features, target_col, weight_col)

fsel.many_to_few(features, target_col, weight_col, nr_to_consider)
if output_dir: fsel.save_state_dict(output_dir)

# get top <nr_to_select> features by mean just as a rule of a thumb
rankings_imp = fsel.get_rankings(True)
rankings_imp["<mean>"] = rankings_imp.mean(axis=1)
rankings_imp.sort_values("<mean>", inplace=True)
top_features = rankings_imp.index.to_list()
rankings_imp.drop("<mean>", axis=1, inplace=True)

# to approximate number of features to consider so
# we end up nr_to_select features when using the less efficient 
# methods

approx_nr_to_select = int(nr_to_select / (corr_threshold+0.001))

fsel.fsel_on_few(top_features[:approx_nr_to_select], target_col, 
                 weight_col, corr_threshold=corr_threshold)
if output_dir: fsel.save_state_dict(output_dir)

rankings = fsel.get_rankings(False)
```

In [30]:
fsel_dir = "./artifacts/dev_combined_fsel_v2"
fsel2 = FeatureSelector(train_df, data_dict=exp_dict)
fsel2.load_state_dict(fsel_dir)
fts = fsel2.get_rankings(True)

In [31]:
fts

,mrmr_shapcq_mc,mrmr_shapcq,mrmr_ivcq,lgbm_shap_214,lgbm_shap_mc_214
p13_alj0416,63,65,2930,159,146
p13_alj5320,66,95,2930,48,28
p13_alj8120,3,3,2930,22,1
p13_all2002,2930,2930,183,2930,2930
p13_all2180,2930,2930,39,214,129
...,...,...,...,...,...
t11_tstu3755,2930,171,2930,2930,2930
t11_tstu4701,34,36,2930,92,56
t11_tstu4704,2930,196,2930,2930,2930
t11_tstu4752,87,89,2930,135,125


#### build base model, set on features

# issue: feature selector did not consider categorical variables.... since we only have < 5 of them, treat manually

#### hyperparam tuning

#### model eval
---
* evaluation segments
    * `weight`
    * around score cut
    * booked, proxy, others

In [9]:
rankings = pd.read_csv("./artifacts/dev1_fsel_v1/ranking.csv", index_col=0)
rankings.shape

(2868, 5)

In [10]:
# hand made shap

import shap
import lightgbm as lgb

print(target, target_indeterminate)
display(train_df[target_indeterminate].value_counts())

default_params = {
 'objective': 'binary',
 'metric': 'auc',
 'boosting': 'gbdt',
 'max_depth': 6,
 'learning_rate': 0.05,
 'min_data_in_leaf': [300],
 'verbosity': -1,
 'seed': 157,
 'n_jobs': 30,
 'n_estimators': 1000
}

lgbm = lgb.LGBMClassifier(**default_params)

list_features = rankings.index.to_list()
trainer = LGBMTrainer()
trainer.train(lgbm, 
              train_df,
              features = list_features,
              target_col = target,
              sample_weight = train_df[weight]
             )
explainer = shap.TreeExplainer(lgbm)
shap_values = explainer.shap_values(train_df[list_features])


target indeterminate


False    440466
Name: indeterminate, dtype: int64

[LightGBM] [Warning] min_data_in_leaf is set=300, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=300
[LightGBM] [Warning] boosting is set=gbdt, boosting_type=gbdt will be ignored. Current value: boosting=gbdt


In [11]:

shap_features = pd.DataFrame(shap_values[1], columns=list(train_df[list_features]))\
                    .apply(lambda x: np.abs(x).mean(), axis=0)\
                    .sort_values(ascending=False)

benchmark_fts = shap_features.index.to_list()[:40]

In [12]:
features = benchmark_fts
from rdsutils.feature_selection.WeightedCorr import WeightedCorr

corr_matrix = WeightedCorr(df=train_df[benchmark_fts+[weight]], wcol=weight)("pearson").abs()
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))
to_drop = [column for column in upper.columns if any(upper[column] > 0.8)]
reduced_shap_features = [f for f in features if f not in (to_drop)]
print(len(reduced_shap_features))

34


In [13]:
len(reduced_shap_features)

34

In [14]:
with open("./artifacts/models/model_params.json", "r") as f:
    model_params = json.load(f)

In [15]:
def get_monotone_dir(woe_dict):
    result = {}
    for k in woe_dict:
        tbl = woe_dict[k]
        if len(tbl) < 2:
            print(k, len(tbl))
        elif tbl.iloc[0]["woe"] < tbl.iloc[1]["woe"]:
            direction = 1
        else:
            direction = -1
        
        result[k] = direction
    return result

with open("./artifacts/dev2_fsel_v2/woe_dict.pkl", "rb") as f:
    woe_dict = pkl.load(f)

monotone_dict = get_monotone_dir(woe_dict)


p13_all9123 1
p13_all9130 1
p13_all9134 1
p13_all9135 1
p13_all9138 1
p13_all9139 1
p13_all9140 1
p13_all9141 1
p13_all9144 1
p13_all9145 1
p13_all9148 1
p13_all9149 1
p13_all9171 1
p13_all9177 1
p13_all9178 1
p13_all9180 1
p13_all9187 1
p13_all9188 1
p13_all9189 1
p13_all9330 1
p13_all9340 1
p13_all9380 1


In [16]:
fts_ = reduced_shap_features[:32]
params_ = copy.deepcopy(model_params["dev2_v2_benchmark"]["params"])

fts_mc = fts_
params_mc = copy.deepcopy(params_)
mc = [monotone_dict[ft] for ft in fts_mc]
params_mc["monotone_constraints"] = mc

In [17]:
combined_v2_benchmark = {"features": fts_,
                     "params": params_,
                     "model_type": "lightgbm"}
combined_v2_benchmark_mc = {"features": fts_mc,
                     "params": params_mc,
                     "model_type": "lightgbm"}
model_params["combined_v2_benchmark"] = combined_v2_benchmark
model_params["combined_v2_benchmark_mc"] = combined_v2_benchmark_mc

In [18]:
model_params.keys()

dict_keys(['dev1_v1_benchmark', 'dev1_v1_benchmark_mc', 'dev1_v1_fsel_32', 'dev1_v1_fsel_32_mc', 'dev2_v2_benchmark', 'dev2_v2_benchmark_mc', 'dev2_v1_benchmark', 'dev2_v1_benchmark_mc', 'combined_v1_benchmark', 'combined_v1_benchmark_mc', 'combined_v2_benchmark', 'combined_v2_benchmark_mc'])

In [21]:
with open("./artifacts/models/model_params.json", "w") as f:
    json.dump(model_params, f)